In [10]:
import pandas as pd
import numpy as np

df= pd.read_csv(r"C:\Users\Priyadharshini\Documents\CNC\aggregated_data.csv")

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from shutil import get_terminal_size
import warnings

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25286 entries, 0 to 25285
Data columns (total 52 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X1_ActualPosition          25286 non-null  float64
 1   X1_ActualVelocity          25286 non-null  float64
 2   X1_ActualAcceleration      25286 non-null  float64
 3   X1_CommandPosition         25286 non-null  float64
 4   X1_CommandVelocity         25286 non-null  float64
 5   X1_CommandAcceleration     25286 non-null  float64
 6   X1_CurrentFeedback         25286 non-null  float64
 7   X1_DCBusVoltage            25286 non-null  float64
 8   X1_OutputCurrent           25286 non-null  float64
 9   X1_OutputVoltage           25286 non-null  float64
 10  X1_OutputPower             25286 non-null  float64
 11  Y1_ActualPosition          25286 non-null  float64
 12  Y1_ActualVelocity          25286 non-null  float64
 13  Y1_ActualAcceleration      25286 non-null  flo

In [13]:
df.columns = df.columns.str.upper()

df = df.drop("EXP_NO", axis=1) # Drop EXP_NO column

# Notes Implementation

df["TARGET"] = df["TARGET"].apply(lambda x: 1 if x == "worn" else 0)

In [14]:
drop_list = ['MACHINING_PROCESS', 'Z1_CURRENTFEEDBACK', 'Z1_DCBUSVOLTAGE', 
             'Z1_OUTPUTCURRENT', 'Z1_OUTPUTVOLTAGE', 'S1_COMMANDACCELERATION', 
             'S1_SYSTEMINERTIA', 'M1_CURRENT_PROGRAM_NUMBER', 'M1_SEQUENCE_NUMBER', 
             'M1_CURRENT_FEEDRATE', "Z1_COMMANDVELOCITY", "Z1_COMMANDACCELERATION"]

df.drop(drop_list, axis=1, inplace=True)

In [19]:
cat_to_numeric = ['FEEDRATE', 'CLAMP_PRESSURE',
                  'Z1_COMMANDVELOCITY', 'Z1_COMMANDACCELERATION', 
                  'Z1_CURRENTFEEDBACK', 'Z1_DCBUSVOLTAGE', 
                  'Z1_OUTPUTCURRENT', 'Z1_OUTPUTVOLTAGE', 'S1_COMMANDACCELERATION', 
                  'S1_SYSTEMINERTIA','M1_CURRENT_FEEDRATE'] 

In [15]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(25286, 39)
##################### Types #####################
X1_ACTUALPOSITION         float64
X1_ACTUALVELOCITY         float64
X1_ACTUALACCELERATION     float64
X1_COMMANDPOSITION        float64
X1_COMMANDVELOCITY        float64
X1_COMMANDACCELERATION    float64
X1_CURRENTFEEDBACK        float64
X1_DCBUSVOLTAGE           float64
X1_OUTPUTCURRENT          float64
X1_OUTPUTVOLTAGE          float64
X1_OUTPUTPOWER            float64
Y1_ACTUALPOSITION         float64
Y1_ACTUALVELOCITY         float64
Y1_ACTUALACCELERATION     float64
Y1_COMMANDPOSITION        float64
Y1_COMMANDVELOCITY        float64
Y1_COMMANDACCELERATION    float64
Y1_CURRENTFEEDBACK        float64
Y1_DCBUSVOLTAGE           float64
Y1_OUTPUTCURRENT          float64
Y1_OUTPUTVOLTAGE          float64
Y1_OUTPUTPOWER            float64
Z1_ACTUALPOSITION         float64
Z1_ACTUALVELOCITY         float64
Z1_ACTUALACCELERATION     float64
Z1_COMMANDPOSITION        float64
S1_A

In [16]:
def grap_column_names(dataframe, categorical_th=10, cardinal_th=20):
    # categorical_cols, categorical_but_cardinal
    categorical_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    numeric_but_categorical = [col for col in dataframe.columns if dataframe[col].nunique() < categorical_th and
                    dataframe[col].dtypes != "O"]
    categorical_but_cardinal = [col for col in dataframe.columns if dataframe[col].nunique() > cardinal_th and
                    dataframe[col].dtypes == "O"]
    categorical_cols = categorical_cols + numeric_but_categorical
    categorical_cols = [col for col in categorical_cols if col not in categorical_but_cardinal]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in numeric_but_categorical]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'categorical_cols: {len(categorical_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'categorical_but_cardinal: {len(categorical_but_cardinal)}')
    print(f'numeric_but_categorical: {len(numeric_but_categorical)}')

    return categorical_cols, num_cols, categorical_but_cardinal

cat_cols, num_cols, cat_but_car = grap_column_names(df)

print("\n\nCategorical Columns: \n", cat_cols)
print("\n\nNumeric Columns: \n", num_cols)
[print("\n\nCategorical but Cardinal EMPTY!!!\n") if cat_but_car == [] else print("Categorical but Cardinal: \n", cat_but_car)]
print("#"*50)

Observations: 25286
Variables: 39
categorical_cols: 3
num_cols: 36
categorical_but_cardinal: 0
numeric_but_categorical: 3


Categorical Columns: 
 ['FEEDRATE', 'CLAMP_PRESSURE', 'TARGET']


Numeric Columns: 
 ['X1_ACTUALPOSITION', 'X1_ACTUALVELOCITY', 'X1_ACTUALACCELERATION', 'X1_COMMANDPOSITION', 'X1_COMMANDVELOCITY', 'X1_COMMANDACCELERATION', 'X1_CURRENTFEEDBACK', 'X1_DCBUSVOLTAGE', 'X1_OUTPUTCURRENT', 'X1_OUTPUTVOLTAGE', 'X1_OUTPUTPOWER', 'Y1_ACTUALPOSITION', 'Y1_ACTUALVELOCITY', 'Y1_ACTUALACCELERATION', 'Y1_COMMANDPOSITION', 'Y1_COMMANDVELOCITY', 'Y1_COMMANDACCELERATION', 'Y1_CURRENTFEEDBACK', 'Y1_DCBUSVOLTAGE', 'Y1_OUTPUTCURRENT', 'Y1_OUTPUTVOLTAGE', 'Y1_OUTPUTPOWER', 'Z1_ACTUALPOSITION', 'Z1_ACTUALVELOCITY', 'Z1_ACTUALACCELERATION', 'Z1_COMMANDPOSITION', 'S1_ACTUALPOSITION', 'S1_ACTUALVELOCITY', 'S1_ACTUALACCELERATION', 'S1_COMMANDPOSITION', 'S1_COMMANDVELOCITY', 'S1_CURRENTFEEDBACK', 'S1_DCBUSVOLTAGE', 'S1_OUTPUTCURRENT', 'S1_OUTPUTVOLTAGE', 'S1_OUTPUTPOWER']


Categorical but C

In [20]:
def cat_summary(dataframe, col_name, plot=False):
    """
    This function shows the frequency of categorical variables.

    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    col_name : str
        The name of the column to be analyzed.
    plot : bool, optional
        The default is False.
    Returns
    -------
    None.
    """
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()

for col in cat_cols:
    cat_summary(df,col)
print("#"*50)
# Drop cols should be Numeric
cat_cols = [col for col in cat_cols if col not in cat_to_numeric]

print(cat_cols)


          FEEDRATE      Ratio
FEEDRATE                     
3.0          13558  53.618603
6.0           5785  22.878273
20.0          3902  15.431464
12.0          1301   5.145140
15.0           740   2.926521
##########################################
                CLAMP_PRESSURE      Ratio
CLAMP_PRESSURE                           
4.0                      11777  46.575180
3.0                       8574  33.908091
2.5                       4935  19.516729
##########################################
        TARGET      Ratio
TARGET                   
1        13308  52.629914
0        11978  47.370086
##########################################
##################################################
['TARGET']


In [ ]:
def target_summary_with_cat(dataframe, target, categorical_col):
    """
    This function shows the mean of the target variable according to the categorical variable.

    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    target : str
        The name of the target variable.
    categorical_col : str
        The name of the categorical variable.
    Returns
    -------
    None.
    """
    print(categorical_col)
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(categorical_col)[target].mean(),
                        "Count": dataframe[categorical_col].value_counts(),
                        "Ratio": 100 * dataframe[categorical_col].value_counts() / len(dataframe)}), end="\n\n\n")

for col in cat_cols:
    target_summary_with_cat(df, "TARGET", col)

print("#"*50)


TARGET
        TARGET_MEAN  Count      Ratio
TARGET                               
0               0.0  11978  47.370086
1               1.0  13308  52.629914


##################################################


In [ ]:
def target_summary_with_num(dataframe, target, numerical_col):
    """
    This function shows the average of numerical variables according to the target variable.
    
    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    target : str
        The name of the target variable.
    numerical_col : str
        The name of the numerical variable.
    Returns
    -------
    None.
    """
    print(dataframe.groupby(target).agg({numerical_col: ["mean", "median", "count"]}), end="\n\n\n")

for col in num_cols:
    target_summary_with_num(df, "TARGET", col)
print("#"*50)

       X1_ACTUALPOSITION              
                    mean median  count
TARGET                                
0             160.924946  154.0  11978
1             157.366321  151.0  13308


       X1_ACTUALVELOCITY              
                    mean median  count
TARGET                                
0              -0.238839    0.0  11978
1              -0.333496    0.0  13308


       X1_ACTUALACCELERATION              
                        mean median  count
TARGET                                    
0                  -0.316067    0.0  11978
1                   0.463586    0.0  13308


       X1_COMMANDPOSITION              
                     mean median  count
TARGET                                 
0              160.925864  154.0  11978
1              157.362940  151.0  13308


       X1_COMMANDVELOCITY              
                     mean median  count
TARGET                                 
0               -0.229884    0.0  11978
1               -0.330953  

In [21]:
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    """
    This function calculates the lower and upper limits for the outliers.

    Calculation:
    Interquantile range = q3 - q1
    Up limit = q3 + 1.5 * interquantile range
    Low limit = q1 - 1.5 * interquantile range

    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    col_name : str
        The name of the column to be analyzed.
    q1 : float, optional
        The default is 0.05.
    q3 : float, optional
        The default is 0.95.
    Returns
    -------
    low_limit, up_limit : float
        The lower and upper limits for the outliers.
    """

    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)

    interquantile_range = quartile3 - quartile1

    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range

    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    """
        This function checks dataframe has outlier or not.

    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    col_name : str
        The name of the column to be analyzed.
    Returns
    -------
    bool
        True if the dataframe has outlier, False otherwise.
    """

    lower_limit, upper_limit = outlier_thresholds(dataframe=dataframe, col_name=col_name)

    if dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)].any(axis=None):
        # print(f'{col_name} have outlier')
        return True
    else:
        return False
    
def replace_with_thresholds(dataframe, variable, q1=0.05, q3=0.95):
    """
    This function replaces the outliers with the lower and upper limits.

    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    variable : str
        The name of the column to be analyzed.
    q1 : float, optional
        The default is 0.05.
    q3 : float, optional
        The default is 0.95.
    Returns 
    -------
    None
    """

    low_limit, up_limit = outlier_thresholds(dataframe, variable, q1, q3)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

for col in num_cols:
    print(f"{col}: {check_outlier(df, col)}")

# Replace with thresholds
for col in num_cols:
    if check_outlier(df, col):
        replace_with_thresholds(df, col)

# Handle the outliers
for col in num_cols:
    print(f"{col}: {check_outlier(df, col)}")

X1_ACTUALPOSITION: False
X1_ACTUALVELOCITY: True
X1_ACTUALACCELERATION: True
X1_COMMANDPOSITION: False
X1_COMMANDVELOCITY: True
X1_COMMANDACCELERATION: True
X1_CURRENTFEEDBACK: True
X1_DCBUSVOLTAGE: True
X1_OUTPUTCURRENT: False
X1_OUTPUTVOLTAGE: True
X1_OUTPUTPOWER: True
Y1_ACTUALPOSITION: False
Y1_ACTUALVELOCITY: True
Y1_ACTUALACCELERATION: True
Y1_COMMANDPOSITION: False
Y1_COMMANDVELOCITY: True
Y1_COMMANDACCELERATION: True
Y1_CURRENTFEEDBACK: True
Y1_DCBUSVOLTAGE: True
Y1_OUTPUTCURRENT: False
Y1_OUTPUTVOLTAGE: True
Y1_OUTPUTPOWER: True
Z1_ACTUALPOSITION: False
Z1_ACTUALVELOCITY: True
Z1_ACTUALACCELERATION: True
Z1_COMMANDPOSITION: False
S1_ACTUALPOSITION: False
S1_ACTUALVELOCITY: False
S1_ACTUALACCELERATION: False
S1_COMMANDPOSITION: False
S1_COMMANDVELOCITY: False
S1_CURRENTFEEDBACK: True
S1_DCBUSVOLTAGE: True
S1_OUTPUTCURRENT: True
S1_OUTPUTVOLTAGE: False
S1_OUTPUTPOWER: True
X1_ACTUALPOSITION: False
X1_ACTUALVELOCITY: False
X1_ACTUALACCELERATION: False
X1_COMMANDPOSITION: False
X1

In [22]:
print(df.isnull().sum())

X1_ACTUALPOSITION         0
X1_ACTUALVELOCITY         0
X1_ACTUALACCELERATION     0
X1_COMMANDPOSITION        0
X1_COMMANDVELOCITY        0
X1_COMMANDACCELERATION    0
X1_CURRENTFEEDBACK        0
X1_DCBUSVOLTAGE           0
X1_OUTPUTCURRENT          0
X1_OUTPUTVOLTAGE          0
X1_OUTPUTPOWER            0
Y1_ACTUALPOSITION         0
Y1_ACTUALVELOCITY         0
Y1_ACTUALACCELERATION     0
Y1_COMMANDPOSITION        0
Y1_COMMANDVELOCITY        0
Y1_COMMANDACCELERATION    0
Y1_CURRENTFEEDBACK        0
Y1_DCBUSVOLTAGE           0
Y1_OUTPUTCURRENT          0
Y1_OUTPUTVOLTAGE          0
Y1_OUTPUTPOWER            0
Z1_ACTUALPOSITION         0
Z1_ACTUALVELOCITY         0
Z1_ACTUALACCELERATION     0
Z1_COMMANDPOSITION        0
S1_ACTUALPOSITION         0
S1_ACTUALVELOCITY         0
S1_ACTUALACCELERATION     0
S1_COMMANDPOSITION        0
S1_COMMANDVELOCITY        0
S1_CURRENTFEEDBACK        0
S1_DCBUSVOLTAGE           0
S1_OUTPUTCURRENT          0
S1_OUTPUTVOLTAGE          0
S1_OUTPUTPOWER      

In [23]:
def high_correlated_cols(dataframe, plot=False, corr_th=0.90):
    """
    This function returns the columns that have a correlation higher than the threshold value.

    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    plot : bool, optional
        The default is False.
    corr_th : float, optional
        The default is 0.90.
    Returns
    -------
    drop_list : list
        The list of columns that have a correlation higher than the threshold value.
    """
    corr = dataframe.corr()
    cor_matrix = corr.abs()
    upper_triangle_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))
    drop_list = [col for col in upper_triangle_matrix.columns if any(upper_triangle_matrix[col] > corr_th)]
    if plot:
        import seaborn as sns
        import matplotlib.pyplot as plt
        sns.set(rc={'figure.figsize': (15, 15)})
        sns.heatmap(corr, cmap="RdBu")
        plt.show()
    return drop_list

df_corr = df.corr()

# f, ax = plt.subplots(figsize=(18, 18))
# sns.heatmap(df_corr, annot=True, fmt=".2f", ax=ax, cmap="magma")
# ax.set_title("Correlation Heatmap", color="black", fontsize=10)
# plt.show()

drop_corr_list=high_correlated_cols(df, plot=False, corr_th=0.95) # 0.95 selected since it is meaningful in statisticaly
print(drop_corr_list)


['X1_COMMANDPOSITION', 'X1_COMMANDVELOCITY', 'Y1_COMMANDPOSITION', 'Y1_COMMANDVELOCITY', 'Z1_COMMANDPOSITION', 'S1_COMMANDPOSITION', 'S1_COMMANDVELOCITY', 'S1_DCBUSVOLTAGE', 'S1_OUTPUTVOLTAGE', 'S1_OUTPUTPOWER']


In [24]:
df.corr()

,X1_ACTUALPOSITION,X1_ACTUALVELOCITY,X1_ACTUALACCELERATION,X1_COMMANDPOSITION,X1_COMMANDVELOCITY,X1_COMMANDACCELERATION,X1_CURRENTFEEDBACK,X1_DCBUSVOLTAGE,X1_OUTPUTCURRENT,X1_OUTPUTVOLTAGE,...,S1_COMMANDPOSITION,S1_COMMANDVELOCITY,S1_CURRENTFEEDBACK,S1_DCBUSVOLTAGE,S1_OUTPUTCURRENT,S1_OUTPUTVOLTAGE,S1_OUTPUTPOWER,FEEDRATE,CLAMP_PRESSURE,TARGET
X1_ACTUALPOSITION,1.000000,-0.030941,-0.016689,0.999897,-0.033107,-0.150383,0.075948,-0.526051,0.349605,-0.297146,...,-0.103584,-0.644630,-0.594439,-0.615841,0.530785,-0.644873,-0.626639,0.453219,0.108508,-0.091920
X1_ACTUALVELOCITY,-0.030941,1.000000,0.030924,-0.028638,0.994133,-0.041841,0.691661,-0.065583,-0.035358,-0.074936,...,0.008428,0.076488,0.052583,0.053917,-0.040347,0.075910,0.061478,-0.023690,-0.018775,-0.010880
X1_ACTUALACCELERATION,-0.016689,0.030924,1.000000,-0.016706,0.046164,0.205882,0.049579,0.042211,-0.004859,0.038058,...,-0.005654,-0.016589,-0.014082,-0.017168,0.012264,-0.016781,-0.014363,0.021513,0.011194,0.006983
X1_COMMANDPOSITION,0.999897,-0.028638,-0.016706,1.000000,-0.030793,-0.150428,0.077517,-0.526255,0.349688,-0.297380,...,-0.103592,-0.644593,-0.594445,-0.615846,0.530807,-0.644838,-0.626626,0.453153,0.108510,-0.092029
X1_COMMANDVELOCITY,-0.033107,0.994133,0.046164,-0.030793,1.000000,0.006160,0.711162,-0.061236,-0.036467,-0.072982,...,0.007471,0.077405,0.052659,0.054002,-0.041120,0.076789,0.062302,-0.022472,-0.018981,-0.011573
X1_COMMANDACCELERATION,-0.150383,-0.041841,0.205882,-0.150428,0.006160,1.000000,0.031428,0.040424,-0.038568,0.028510,...,-0.005118,0.021843,0.022230,0.020908,-0.024701,0.021894,0.023535,0.018989,0.009716,0.004172
X1_CURRENTFEEDBACK,0.075948,0.691661,0.049579,0.077517,0.711162,0.031428,1.000000,-0.017293,-0.076098,-0.016464,...,0.006329,0.108960,0.090905,0.092608,-0.078877,0.108388,0.097353,-0.015090,-0.013575,-0.020942
X1_DCBUSVOLTAGE,-0.526051,-0.065583,0.042211,-0.526255,-0.061236,0.040424,-0.017293,1.000000,-0.440002,0.768037,...,0.089914,0.556048,0.514710,0.531699,-0.524784,0.556337,0.547260,-0.269666,-0.045534,0.077357
X1_OUTPUTCURRENT,0.349605,-0.035358,-0.004859,0.349688,-0.036467,-0.038568,-0.076098,-0.440002,1.000000,-0.429552,...,-0.093562,-0.596049,-0.578973,-0.603326,0.727447,-0.599540,-0.611868,-0.172817,-0.308918,-0.080973
X1_OUTPUTVOLTAGE,-0.297146,-0.074936,0.038058,-0.297380,-0.072982,0.028510,-0.016464,0.768037,-0.429552,1.000000,...,0.068572,0.307827,0.297941,0.308500,-0.310773,0.309724,0.316659,0.024928,0.059563,0.076638


In [25]:
#Encoding
cat_cols, num_cols, cat_but_car = grap_column_names(df)

for col in cat_cols:
    cat_cols = [col for col in cat_cols if col not in cat_to_numeric]

for col in cat_to_numeric:
    num_cols.append(col)


cat_cols = [col for col in cat_cols if col not in ["TARGET"]]
print(cat_cols)

Observations: 25286
Variables: 39
categorical_cols: 3
num_cols: 36
categorical_but_cardinal: 0
numeric_but_categorical: 3
[]


In [26]:
def one_hot_encoder(dataframe, categorical_columns, drop_first=True):
    """
    This function encodes the categorical variables to numericals.

    Parameters
    ----------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    categorical_columns : list
        The name of the column to be encoded.
    drop_first : bool, optional
        Dummy trap. The default is True.
    Returns
    -------
    dataframe : pandas dataframe
        The dataframe to be analyzed.
    """
    dataframe = pd.get_dummies(dataframe, columns=categorical_columns, drop_first=drop_first)
    return dataframe

df = one_hot_encoder(df, cat_cols, drop_first=True)

print(df.head())


   X1_ACTUALPOSITION  X1_ACTUALVELOCITY  X1_ACTUALACCELERATION  \
0              198.0                0.0                   0.00   
1              198.0              -10.8                -350.00   
2              196.0              -17.8                  -6.25   
3              194.0              -18.0                   0.00   
4              193.0              -17.9                 -18.80   

   X1_COMMANDPOSITION  X1_COMMANDVELOCITY  X1_COMMANDACCELERATION  \
0               198.0                 0.0                0.000000   
1               198.0               -13.6              -11.608750   
2               196.0               -17.9               -0.000095   
3               194.0               -17.9               -0.000095   
4               192.0               -17.9                0.000095   

   X1_CURRENTFEEDBACK  X1_DCBUSVOLTAGE  X1_OUTPUTCURRENT  X1_OUTPUTVOLTAGE  \
0                0.18           0.0207             329.0              2.77   
1              -10.90           

In [28]:
df


,X1_ACTUALPOSITION,X1_ACTUALVELOCITY,X1_ACTUALACCELERATION,X1_COMMANDPOSITION,X1_COMMANDVELOCITY,X1_COMMANDACCELERATION,X1_CURRENTFEEDBACK,X1_DCBUSVOLTAGE,X1_OUTPUTCURRENT,X1_OUTPUTVOLTAGE,...,S1_COMMANDPOSITION,S1_COMMANDVELOCITY,S1_CURRENTFEEDBACK,S1_DCBUSVOLTAGE,S1_OUTPUTCURRENT,S1_OUTPUTVOLTAGE,S1_OUTPUTPOWER,FEEDRATE,CLAMP_PRESSURE,TARGET
0,198.0,0.00,0.00,198.0,0.00,0.000000,0.18,0.0207,329.0,2.77,...,-361.0,0.0,0.524,2.740000e-19,329.0,0.0,6.960000e-07,6.0,4.0,0
1,198.0,-10.80,-350.00,198.0,-13.60,-11.608750,-10.90,0.1860,328.0,23.30,...,-361.0,0.0,-0.288,2.740000e-19,328.0,0.0,-5.270000e-07,6.0,4.0,0
2,196.0,-17.80,-6.25,196.0,-17.90,-0.000095,-8.59,0.1400,328.0,30.60,...,-361.0,0.0,0.524,2.740000e-19,328.0,0.0,9.100000e-07,6.0,4.0,0
3,194.0,-18.00,0.00,194.0,-17.90,-0.000095,-6.11,0.1300,327.0,30.30,...,-361.0,0.0,-0.180,2.740000e-19,328.0,0.0,1.070000e-06,6.0,4.0,0
4,193.0,-17.90,-18.80,192.0,-17.90,0.000095,-5.70,0.1140,328.0,30.50,...,-361.0,0.0,0.226,2.740000e-19,328.0,0.0,1.960000e-06,6.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25281,144.0,-2.30,100.00,144.0,-2.57,0.977000,-5.05,0.0732,327.0,9.56,...,-72.0,53.3,20.000,8.260000e-01,323.0,118.0,1.580000e-01,3.0,2.5,1
25282,144.0,-2.47,25.00,144.0,-2.46,1.080000,-5.62,0.0996,327.0,10.30,...,-66.6,53.3,18.800,9.710000e-01,320.0,119.0,1.870000e-01,3.0,2.5,1
25283,143.0,-2.20,81.20,143.0,-2.35,1.170000,-4.42,0.0539,327.0,7.79,...,-61.3,53.3,24.400,9.500000e-01,322.0,119.0,1.820000e-01,3.0,2.5,1
25284,143.0,-2.05,68.70,143.0,-2.23,1.260000,-5.83,0.0809,327.0,8.32,...,-56.0,53.3,21.600,9.590000e-01,322.0,118.0,1.820000e-01,3.0,2.5,1


In [30]:
# 6. Save the Dataset
df.to_csv('aggregated_cleaned.csv', index=False)